# Azure AI Agents s podrškom za Model Context Protocol (MCP) - Python

Ovaj notebook pokazuje kako koristiti Azure AI Agents s alatima za Model Context Protocol (MCP) u Pythonu. Prikazuje kako stvoriti inteligentnog agenta koji može koristiti vanjske MCP servere (poput Microsoft Learn) za poboljšane mogućnosti koristeći autentifikaciju bez ključeva.


## Instalirajte potrebne Python pakete

Prvo, trebamo instalirati potrebne Python pakete:
- **azure-ai-projects**: Osnovni SDK za Azure AI Projects
- **azure-ai-agents**: SDK za Azure AI Agents za kreiranje i upravljanje agentima
- **azure-identity**: Omogućuje autentifikaciju bez ključeva koristeći DefaultAzureCredential
- **mcp**: Implementacija Model Context Protocol za Python


## Prednosti autentifikacije bez ključeva

Ovaj bilježnik demonstrira **autentifikaciju bez ključeva**, koja pruža nekoliko prednosti:
- ✅ **Nema upravljanja API ključevima** - Koristi autentifikaciju temeljenu na Azure identitetu
- ✅ **Poboljšana sigurnost** - Nema pohranjenih tajni u kodu ili konfiguracijskim datotekama
- ✅ **Automatska rotacija vjerodajnica** - Azure upravlja životnim ciklusom vjerodajnica
- ✅ **Kontrola pristupa temeljena na ulogama** - Koristi Azure RBAC za precizno definirane dozvole
- ✅ **Podrška za više okruženja** - Radi besprijekorno u razvojnim i produkcijskim okruženjima

`DefaultAzureCredential` automatski odabire najbolji dostupni izvor vjerodajnica:
1. **Upravljani identitet** (kada se pokreće u Azureu)
2. **Azure CLI** vjerodajnice (tijekom lokalnog razvoja)
3. **Visual Studio** vjerodajnice
4. **Varijable okruženja** (ako su konfigurirane)
5. **Autentifikacija putem interaktivnog preglednika** (kao rezervna opcija)


## Postavljanje autentifikacije bez ključeva

**Preduvjeti za autentifikaciju bez ključeva:**

### Za lokalni razvoj:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Za Azure okruženja:
- Omogućite **System-assigned Managed Identity** na svom Azure resursu
- Dodijelite odgovarajuće **RBAC uloge** za upravljanu identitet:
  - `Cognitive Services OpenAI User` za pristup Azure OpenAI
  - `AI Developer` za pristup Azure AI Projects

### Varijable okruženja (Opcionalno):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**Nisu potrebni API ključevi ili veze!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Uvoz potrebnih knjižnica

Uvezite potrebne Python module:
- **os, time**: Standardne Python knjižnice za varijable okruženja i kašnjenja
- **AIProjectClient**: Glavni klijent za Azure AI projekte
- **DefaultAzureCredential**: Autentifikacija bez ključa za Azure usluge
- **Klase povezane s MCP-om**: Za kreiranje i upravljanje MCP alatima te rukovanje odobrenjima


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Konfiguriranje postavki MCP poslužitelja

Postavite konfiguraciju MCP poslužitelja koristeći varijable okruženja s rezervnim zadanim vrijednostima:
- **MCP_SERVER_URL**: URL MCP poslužitelja (zadano je Microsoft Learn API)
- **MCP_SERVER_LABEL**: Oznaka za identifikaciju MCP poslužitelja (zadano je "mslearn")

Ovaj pristup omogućuje fleksibilnu konfiguraciju u različitim okruženjima.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Kreiranje Azure AI Project klijenta (Autentifikacija bez ključa)

Inicijalizirajte Azure AI Project klijent koristeći **autentifikaciju bez ključa**:
- **endpoint**: URL krajnje točke Azure AI Foundry projekta
- **credential**: Koristi `DefaultAzureCredential()` za sigurnu autentifikaciju bez ključa
- **Nema potrebe za API ključevima**: Automatski otkriva i koristi najbolju dostupnu vjerodajnicu

**Proces autentifikacije:**
1. Provjerava Managed Identity (u Azure okruženjima)
2. Prelazi na Azure CLI vjerodajnice (za lokalni razvoj)
3. Koristi druge dostupne izvore vjerodajnica po potrebi

Ovaj pristup eliminira potrebu za upravljanjem API ključevima ili povezanim stringovima u vašem kodu.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Kreirajte MCP alat

Kreirajte MCP alat koji se povezuje s Microsoft Learn MCP serverom:
- **server_label**: Identifikator za MCP server
- **server_url**: URL krajnja točka MCP servera
- **allowed_tools**: Opcionalni popis za ograničavanje alata koji se mogu koristiti (prazan popis omogućuje sve alate)

Ovaj alat omogućit će agentu pristup Microsoft Learn dokumentaciji i resursima.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Kreirajte agenta i provedite razgovor (radni tijek bez ključeva)

Ovaj detaljni odjeljak prikazuje kompletan **radni tijek agenta bez ključeva**:

1. **Kreirajte AI agenta**: Postavite agenta s GPT-4.1 nano modelom i MCP alatima  
2. **Kreirajte nit**: Uspostavite nit za komunikaciju  
3. **Pošaljite poruku**: Pitajte agenta o razlikama između Azure OpenAI i OpenAI  
4. **Rukovanje odobrenjima alata**: Automatski odobrite pozive MCP alata kada je potrebno  
5. **Pratite izvršenje**: Pratite napredak agenta i poduzmite potrebne radnje  
6. **Prikažite rezultate**: Prikaz razgovora i detalja o korištenju alata  

**Značajke bez ključeva:**  
- ✅ **Bez hardkodiranih tajni** - Sva autentifikacija obavlja se putem Azure identiteta  
- ✅ **Sigurno prema zadanim postavkama** - Koristi kontrolu pristupa temeljenu na ulogama  
- ✅ **Pojednostavljeno postavljanje** - Nema potrebe za upravljanjem vjerodajnicama  
- ✅ **Prikladno za reviziju** - Sav pristup prati se putem Azure identiteta  

Agent će koristiti MCP alate za pristup Microsoft Learn resursima uz potpunu sigurnost i bez upravljanja API ključevima.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati autoritativnim izvorom. Za kritične informacije preporučuje se profesionalni prijevod od strane ljudskog prevoditelja. Ne preuzimamo odgovornost za bilo kakva nesporazuma ili pogrešna tumačenja koja proizlaze iz korištenja ovog prijevoda.
